## MVP Siamese LSTM Net

This is a baseline siamese LSTM net. The purpose is to build out the architecture, and see if the net can get as good as validation score as the classifiers.

Ideas Implemented:
* Increased Dropout rate to 50%

In [1]:
# data manipulation
import utils
import pandas as pd
import numpy as np
import logging

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input, Add, concatenate, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping
from keras.models import load_model

from sklearn import metrics
from sklearn.model_selection import train_test_split

# plotting
from IPython.display import SVG

Using TensorFlow backend.


In [2]:
X_train = utils.load('X_train')
y_train = utils.load('y_train')
model_name = 'lstm_dropout_50'

## Tokenize and Encode vocabulary

1. Limit the vocab to 20,000 words.
2. Clean questions only and do not lemmatize.
3. Limit the question length to 100 tokens.

In [3]:
vocabulary_size = 20000
max_q_len = 100

X_train_stack = utils.clean_questions(utils.stack_questions(X_train))

tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train_stack)

sequences = tokenizer.texts_to_sequences(X_train_stack)
data = pad_sequences(sequences, maxlen=max_q_len)

print(data.shape)
data[:,0].sum()

(606398, 100)


7212

## Embedding Matrix

1. Calculates the embedding matrix utilizing spaCy `en_core_web_lg` word vectors.
  * https://spacy.io/models/en#en_core_web_lg
  * GloVe vectors trained on Common Crawl

In [4]:
try:
    embedding_matrix = utils.load('embedding_matrix')
except:
    # create a weight matrix for words in training docs
    embedding_matrix = np.zeros((vocabulary_size, 300))
    for word, index in tokenizer.word_index.items():
    #     print(word, index, end='\r')
        if index > vocabulary_size - 1:
            break
        else:
            embedding_vector = utils.nlp(word).vector
            if embedding_vector is not None:
                embedding_matrix[index] = embedding_vector
    #     break

    utils.save(embedding_matrix, 'embedding_matrix')

## Define the batch to pass into the network

Create arrays to split the stacked data into question 1 set and question 2 set for each pair.

**Need to cleanup this cell in the next model**

In [5]:
# cooncatenate the two questions
odd_idx = [i for i in range(data.shape[0]) if i % 2 == 1]
even_idx = [i for i in range(data.shape[0]) if i % 2 == 0]

data_1 = data[odd_idx]
data_2 = data[odd_idx]

# split the data set into a validation set
data_train, data_val, label_train, label_val = train_test_split(np.hstack([data_1, data_2]), 
                                                                y_train, 
                                                                stratify=y_train, 
                                                                test_size = 0.33,
                                                                random_state=42)

# split the concatenation back into 2 data sets for the siamese network
data_1_train = data_train[:, :max_q_len]
data_2_train = data_train[:, max_q_len:]
data_1_val = data_val[:, :max_q_len]
data_2_val = data_val[:, max_q_len:]

print(f'Train major class: {len(label_train[label_train == 0]) / len(label_train):.2}')
print(f'Val major class: {len(label_val[label_val == 0]) / len(label_val):.2}')

Train major class: 0.63
Val major class: 0.63


## Build out legs of the siamese network

The architecure is the following,

0. Input - (100,) word tensor
1. Embedding Layer - outputs (300,) **not trainable**
2. LSTM - default outputs (300,)
3. Concatenate the two nets outputs (600,)
4. BatchNormalization
5. Dropout - 20%
6. Dense - outputs (100,), activation `tanh` -- somewhat random decision
7. BatchNormalization
8. Dropout - 20%
9. Dense - outputs (1,), activation `sigmoid`

In [6]:
# Creating word embedding layer
embedding_layer = Embedding(vocabulary_size, 300, input_length=100, 
                                     weights=[embedding_matrix], trainable=False)

# Creating LSTM Encoder
# Bidirectional(LSTM(self.number_lstm_units, dropout=self.rate_drop_lstm, recurrent_dropout=self.rate_drop_lstm))
lstm_layer = LSTM(300)

# Creating LSTM Encoder layer for First Sentence
sequence_1_input = Input(shape=(100,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

# Creating LSTM Encoder layer for Second Sentence
sequence_2_input = Input(shape=(100,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
x2 = lstm_layer(embedded_sequences_2)



In [7]:
# Merging two LSTM encodes vectors from sentences to
# pass it to dense layer applying dropout and batch normalisation

merged = concatenate([x1, x2])
# merged = BatchNormalization()(merged)
merged = Dropout(.5)(merged)
merged = Dense(100, activation='tanh')(merged)
# merged = BatchNormalization()(merged)
merged = Dropout(0.5)(merged)
preds = Dense(1, activation='sigmoid')(merged)

model = Model(inputs=[sequence_1_input, sequence_2_input], outputs=preds)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
# SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     6000000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 300)          721200      embedding_1[0][0]                
          

In [8]:
# Callbacks

file_path = '../data/keras_models/' + model_name + '_{epoch:02d}-{val_loss:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(filepath=file_path, save_best_only=True)


tensorboard = TensorBoard(log_dir='../data/tensorboard')

early_stopping = EarlyStopping(monitor='val_loss', 
                               min_delta=0, 
                               patience=3, 
                               verbose=1, 
                               mode='auto', 
                               restore_best_weights=True)

# calc_auc = IntervalEvaluation(([data_1_val, data_2_val], label_val), interval=1)

In [9]:
model.fit([data_1_train, data_2_train], label_train, 
          validation_data=([data_1_val, data_2_val], label_val),
                  epochs=200, batch_size=128, shuffle=True,
                  callbacks=[model_checkpoint, tensorboard, early_stopping])

Train on 203143 samples, validate on 100056 samples
Epoch 1/200
203143/203143 [==============================] - 557s 3ms/step - loss: 0.5740 - acc: 0.7015 - val_loss: 0.5723 - val_acc: 0.7130
Epoch 2/200
203143/203143 [==============================] - 556s 3ms/step - loss: 0.5301 - acc: 0.7373 - val_loss: 0.5376 - val_acc: 0.7305
Epoch 3/200
203143/203143 [==============================] - 553s 3ms/step - loss: 0.5022 - acc: 0.7566 - val_loss: 0.5267 - val_acc: 0.7454
Epoch 4/200
203143/203143 [==============================] - 543s 3ms/step - loss: 0.4715 - acc: 0.7770 - val_loss: 0.5229 - val_acc: 0.7505
Epoch 5/200
203143/203143 [==============================] - 542s 3ms/step - loss: 0.4348 - acc: 0.8000 - val_loss: 0.5218 - val_acc: 0.7518
Epoch 6/200
203143/203143 [==============================] - 541s 3ms/step - loss: 0.3959 - acc: 0.8229 - val_loss: 0.5421 - val_acc: 0.7517
Epoch 7/200
203143/203143 [==============================] - 542s 3ms/step - loss: 0.3586 - acc: 0.843

## Results

In [11]:
# model = load_model('../data/keras_models/mvp_batch_norm08-0.54.hdf5')

y_prob = model.predict([data_1_val, data_2_val], batch_size=128, verbose=1)

100056/100056 [==============================] - 88s 877us/step


In [12]:
results_df = utils.load('results')

results_df = results_df.drop(index=model_name, errors='ignore')
results_df = results_df.append(utils.log_keras_scores(label_val, y_prob, model_name))
results_df.sort_values('avg_auc', ascending=False)


,avg_accuracy,std_accuracy,avg_precision,std_precision,avg_recall,std_recall,avg_f1,std_f1,avg_auc,std_auc,avg_log_loss,std_log_loss
rf_feat_eng_model_lemma_clean,0.783667,0.002260,0.708853,0.003681,0.702725,0.001666,0.705774,0.002658,0.868202,0.001148,0.436197,0.000640
ensemble_rf_xgb,0.779000,0.002740,0.697794,0.004357,0.708157,0.001912,0.702935,0.003148,0.863334,0.001438,0.441784,0.001107
feat_eng_model_lemma_clean,0.763927,0.002404,0.676166,0.003904,0.692113,0.001128,0.684044,0.002549,0.846923,0.001643,0.456929,0.001410
feat_eng_model_lemma_fix,0.744356,0.002107,0.664513,0.004333,0.621357,0.000901,0.642201,0.001609,0.822197,0.001710,0.488131,0.001342
feat_eng_model,0.743614,0.002021,0.664102,0.003502,0.618400,0.001553,0.640434,0.002281,0.821070,0.001428,0.489465,0.001141
ensemble_rf_xgb_cos_sim,0.738700,0.007359,0.661290,0.010948,0.612827,0.009669,0.636128,0.009994,0.819987,0.005193,0.493703,0.003901
lstm_dropout_50,0.751849,0.000000,0.690400,0.000000,0.594510,0.000000,0.638877,0.000000,0.802315,0.000000,8.570912,0.000000
lstm_mvp,0.749760,0.000000,0.685627,0.000000,0.595133,0.000000,0.637183,0.000000,0.801019,0.000000,8.643059,0.000000
cos_sim_tfidf_model,0.729511,0.001216,0.661680,0.002219,0.547188,0.001744,0.599010,0.001703,0.800271,0.001291,0.512085,0.001299
lstm_dropout50_dense50_BatchNorm,0.728612,0.000000,0.682992,0.000000,0.494492,0.000000,0.573654,0.000000,0.772344,0.000000,9.373478,0.000000


In [13]:
utils.save(results_df, 'results')

### Next Steps

Slightly better. Let's now apply a dropout rates to the LSTM layer.